Trong đoạn mã của bạn, các phần có thể dẫn đến kết quả **không nhất quán** giữa các lần chạy là do sử dụng các thuật toán hoặc thư viện có thành phần **random** (ngẫu nhiên). Dưới đây là các phần trong mã liên quan đến sự ngẫu nhiên và cách khắc phục:

---

## **1. Gaussian Mixture Model (GMM)**

- **Ngẫu nhiên trong GMM:**
  GMM sử dụng các phương pháp ngẫu nhiên để khởi tạo cụm (ví dụ, khởi tạo centroids) và quá trình tối ưu hóa. Điều này được điều khiển bởi tham số `random_state`.

- **Vấn đề:**
  Nếu `random_state` không được cố định hoặc được đặt khác nhau ở từng lần chạy, kết quả của việc phân cụm sẽ không giống nhau.

- **Giải pháp:**
  Đảm bảo `random_state` được cố định khi khởi tạo mô hình GMM.

### **Ví dụ trong mã của bạn:**
```python
gm = GaussianMixture(n_components=n, random_state=random_state)  # Cố định random_state
```

---

## **2. UMAP**

- **Ngẫu nhiên trong UMAP:**
  UMAP sử dụng ngẫu nhiên trong quá trình khởi tạo bố cục không gian nhúng (embeddings) và xây dựng đồ thị lân cận. Tham số `random_state` kiểm soát sự ngẫu nhiên này.

- **Vấn đề:**
  Nếu `random_state` không được cố định, các lần chạy UMAP sẽ tạo ra kết quả khác nhau.

- **Giải pháp:**
  Sử dụng tham số `random_state` trong UMAP để cố định kết quả.

### **Ví dụ trong mã của bạn:**
```python
umap.UMAP(
    n_neighbors=n_neighbors,
    n_components=dim,
    metric=metric,
    random_state=RANDOM_SEED  # Sử dụng RANDOM_SEED cố định
).fit_transform(embeddings)
```

---

## **3. Lựa Chọn Số Lượng Cụm (Optimal Clusters)**

- **Ngẫu nhiên trong BIC và GMM:**
  Hàm `get_optimal_clusters` sử dụng GMM nhiều lần với các số cụm khác nhau. Mỗi lần GMM chạy, nó có thể tạo ra kết quả khác nhau nếu `random_state` không được cố định.

- **Vấn đề:**
  Nếu không cố định `random_state`, giá trị BIC sẽ dao động giữa các lần chạy, dẫn đến việc chọn số cụm khác nhau.

- **Giải pháp:**
  Đảm bảo GMM trong hàm `get_optimal_clusters` cũng được cố định `random_state`.

### **Ví dụ trong mã của bạn:**
```python
gm = GaussianMixture(n_components=n, random_state=random_state)  # Cố định random_state trong vòng lặp
```

---

## **4. Tổng Kết Các Phần Random**

| **Vị Trí**                    | **Ngẫu Nhiên Do**                              | **Tham Số Cần Kiểm Soát** | **Giải Pháp**                                         |
|-------------------------------|-----------------------------------------------|---------------------------|------------------------------------------------------|
| **Gaussian Mixture Model**    | Khởi tạo centroids, tối ưu cụm                | `random_state`            | Đặt `random_state` khi khởi tạo GMM.                |
| **UMAP**                      | Khởi tạo bố cục không gian nhúng              | `random_state`            | Đặt `random_state` khi gọi UMAP.                    |
| **get_optimal_clusters**      | GMM chạy nhiều lần cho các số cụm khác nhau   | `random_state`            | Cố định `random_state` trong mỗi lần khởi tạo GMM.  |

---

## **5. Cách Cố Định Tất Cả Random**

Để đảm bảo tính tái lập hoàn toàn, hãy đảm bảo:
1. Sử dụng một giá trị `random_state` cố định (ví dụ `RANDOM_SEED = 224`) ở mọi nơi có ngẫu nhiên.
2. Kiểm tra kỹ các thuật toán hoặc thư viện có sử dụng ngẫu nhiên và truyền tham số `random_state` tương ứng.

**Ví dụ trong đoạn mã của bạn:**
```python
RANDOM_SEED = 224  # Seed cố định

def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors,
        n_components=dim,
        metric=metric,
        random_state=RANDOM_SEED  # Cố định random_state
    ).fit_transform(embeddings)

def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)  # Cố định random_state
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]
```

---

Nếu sau khi sửa mã vẫn gặp vấn đề với tính ngẫu nhiên, bạn hãy kiểm tra xem có thư viện nào khác ngoài UMAP và GMM đang sử dụng random không nhé!